In [ ]:
import os
import pandas as pd
import string
import zipfile
import re
import numpy as np
#! pip install langdetect
#from langdetect import detect

#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords

In [ ]:
def extract_zip(zip_file):
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall()
        print("Zip file extracted successfully.")
    except zipfile.BadZipFile:
        print("Error: Not a valid zip file.")
    except FileNotFoundError:
        print("Error: File not found.")

zip_file = "data.zip"
extract_zip(zip_file)

Zip file extracted successfully.


In [ ]:
def read_poem(file_path):
  file_path = file_path.replace('\\', '/')
  with open(file_path, 'r') as file:
    lines = file.readlines()
    poem = ' '.join(lines).replace('\n', '')
  return poem

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
X = df['Poem'].apply(read_poem).values
y = df['Title'].apply(lambda txt: re.sub(r'[^A-Za-z0-9\s]', '', txt).replace("\u200a", "").strip()).values
stratify = df['Category'].values

In [ ]:
idx = []
for i in range(len(X)) :
  if len(X[i]) <= 100 : idx.append(i)

X = np.delete(X, idx)
y = np.delete(y, idx)
stratify = np.delete(stratify, idx)

idx = []
for i in range(len(y)) :
  if len(y[i]) <= 4 : idx.append(i)

X = np.delete(X, idx)
y = np.delete(y, idx)
stratify = np.delete(stratify, idx)

In [ ]:
len(X)

3129

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77, stratify=stratify)
train_df = pd.DataFrame({'source_text': X_train, 'target_text': y_train})
eval_df = pd.DataFrame({'source_text': X_test, 'target_text': y_test})

In [ ]:
# It's advisable to create a new python environment and install simplet5
!pip install --upgrade simplet5

In [ ]:
# import
from simplet5 import SimpleT5

# instantiate
model = SimpleT5()

# load (supports t5, mt5, byT5 and CodeT5 models)
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 900,
            target_max_token_len = 5,
            batch_size = 8,
            max_epochs = 5,
            use_gpu = True,
            outputdir = "outputs_2",
            early_stopping_patience_epochs = 0,
            precision = 16
            )

In [ ]:
# load trained T5 model
model.load_model("t5","/content/outputs/simplet5-epoch-4-train-loss-nan-val-loss-2.9005", use_gpu=True)

In [ ]:
# Enchanted Beach with Three Fluid Graces
model.predict("All things begin from the spindle we say life spun from graces I grew like rain from rumbles of my parents cloud a dark dawn admitting growth In the beginning I squeeze into existence with a head white silhouette like formed from the stomach of the hard earth on a farther landscape face formed in a hole a body merging into visibility The last war of my body was unscathed admitting the shapes of various dances The second grace where I am a boy seeking existential relevance my visage like a landscape with an equestrian man with the body of dust pursuing the horse of his identity relevance seeking The thread of my life stretched to the doorsill of disappearance the length of my language nearing a silence I am the last body rhyming with the earth becoming more illusion than truth Holding onto the edge of my thread bowing to the ache of an empty body my fingers grip the edge of my cloth Nature becomes a skull".lower())

['from graces']

In [ ]:
# Caged Bird
model.predict("The free bird leaps on the back of the wind and floats downstream till the current ends and dips his wings in the orange sun rays and dares to claim the sky But a bird that stalks down his narrow cage can seldom see through his bars of ragehis wings are clipped and his feet are tied so he opens his throat to sing The caged bird sings with fearful trill of the things unknown but longed for still and his tune is heard on the distant hill for the caged bird sings of freedom The free bird thinks of another breeze and the trade winds soft through the sighing trees and the fat worms waiting on a dawn-bright lawn and he names the sky his own But a caged bird stands on the grave of dreams his shadow shouts on a nightmare scream his wings are clipped and his feet are tied so he opens his throat to sing The caged bird sings with a fearful trill of things unknown but longed for still and his tune is heard on the distant hill for the caged bird sings of freedom".lower())

['a caged bird']

In [ ]:
# Each Life Converges to Some Centre
model.predict("Each life converges to some centre Expressed or still Exists in every human nature A goal Admitted scarcely to itself it may be Too fair For credibilitys temerity To dare Adored with caution as a brittle heaven To reach Were hopeless as the rainbows raiment To touch Yet persevered toward surer for the distance How high Unto the saints slow diligence The sky Ungained it may be by a lifes low venture But then Eternity enables the endeavoring Again".lower())

['A Lifes Low Venture']

In [ ]:
def clean(input_string):
    punctuation_chars = set(string.punctuation)
    cleaned_string = ''.join(char for char in input_string if char not in punctuation_chars)
    cleaned_string = cleaned_string.replace('\n', '')
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string)
    return cleaned_string.lower()

In [ ]:
# If ---
poem = """
  If you can keep your head when all about you
  Are losing theirs and blaming it on you;
  If you can trust yourself when all men doubt you,
  But make allowance for their doubting too:
  If you can wait and not be tired by waiting,
  Or, being lied about, don’t deal in lies,
  Or being hated don’t give way to hating,
  And yet don’t look too good, nor talk too wise;
  If you can dream—and not make dreams your master;
  If you can think—and not make thoughts your aim,
  If you can meet with Triumph and Disaster
  And treat those two impostors just the same:
  If you can bear to hear the truth you’ve spoken
  Twisted by knaves to make a trap for fools,
  Or watch the things you gave your life to, broken,
  And stoop and build ’em up with worn-out tools;
  If you can make one heap of all your winnings
  And risk it on one turn of pitch-and-toss,
  And lose, and start again at your beginnings,
  And never breathe a word about your loss:
  If you can force your heart and nerve and sinew
  To serve your turn long after they are gone,
  And so hold on when there is nothing in you
  Except the Will which says to them: ‘Hold on!’
  If you can talk with crowds and keep your virtue,
  Or walk with Kings—nor lose the common touch,
  If neither foes nor loving friends can hurt you,
  If all men count with you, but none too much:
  If you can fill the unforgiving minute
  With sixty seconds’ worth of distance run,
  Yours is the Earth and everything that’s in it,
  And—which is more—you’ll be a Man, my son!
"""
model.predict(clean(poem))

['if you can keep your head when all about you are losing theirs']

In [ ]:
# Hope is the thing with feathers
poem = """
  Hope is the thing with feathers
  That perches in the soul,
  And sings the tune without the words,
  And never stops at all,
  And sweetest in the gale is heard;
  And sore must be the storm
  That could abash the little bird
  That kept so many warm.
  I’ve heard it in the chillest land,
  And on the strangest sea;
  Yet, never, in extremity,
  It asked a crumb of me.
"""
model.predict(clean(poem))

['Hope is the thing with feathers']

In [ ]:
# The Guest House
poem = """
  This being human is a guest house.
  Every morning a new arrival.
  A joy, a depression, a meanness,
  some momentary awareness comes
  As an unexpected visitor.
  Welcome and entertain them all!
  Even if they’re a crowd of sorrows,
  who violently sweep your house
  empty of its furniture,
  still treat each guest honorably.
  He may be clearing you out
  for some new delight.
  The dark thought, the shame, the malice,
  meet them at the door laughing,
  and invite them in.
  Be grateful for whoever comes,
  because each has been sent
  as a guide from beyond.
"""
model.predict(clean(poem))

['a guest house every morning']

In [ ]:
# Life Is
poem = """
  Life is an opportunity, benefit from it.
  Life is beauty, admire it.
  Life is a dream, realize it.
  Life is a challenge, meet it.
  Life is a duty, complete it.
  Life is a game, play it.
  Life is a promise, fulfill it.
  Life is sorrow, overcome it.
  Life is a song, sing it.
  Life is a struggle, accept it.
  Life is a tragedy, confront it.
  Life is an adventure, dare it.
  Life is luck, make it.
  Life is life, fight for it.
"""
model.predict(clean(poem))

['life is an opportunity']

In [ ]:
# Start Where You Stand
poem = """
  Start where you stand and never mind the past,
  The past won’t help you in beginning new,
  If you have left it all behind at last
  Why, that’s enough, you’re done with it, you’re through;
  This is another chapter in the book,
  This is another race that you have planned,
  Don’t give the vanished days a backward look,
  Start where you stand.
  The world won’t care about your old defeats
  If you can start anew and win success,
  The future is your time, and time is fleet
  And there is much of work and strain and stress;
  Forget the buried woes and dead despairs,
  Here is a brand new trial right at hand,
  The future is for him who does and dares,
  Start where you stand.
  Old failures will not halt, old triumphs aid,
  To-day’s the thing, to-morrow soon will be;
  Get in the fight and face it unafraid,
  And leave the past to ancient history;
  What has been, has been; yesterday is dead
  And by it you are neither blessed nor banned,
  Take courage, man, be brave and drive ahead,
  Start where you stand.
"""
model.predict(clean(poem))

['start where you stand']

In [ ]:
# Don’t Quit
poem = """
  When things go wrong, as they sometimes will,
  when the road you’re trudging seems all uphill,
  when the funds are low and the debts are high,
  and you want to smile but you have to sigh,
  when care is pressing you down a bit – rest if you must, but don’t you quit.
  Life is queer with its twists and turns.
  As everyone of us sometimes learns.
  And many a fellow turns about when he might have won had he stuck it out.
  Don’t give up though the pace seems slow – you may succeed with another blow.
  Often the goal is nearer than it seems to a faint and faltering man;
  Often the struggler has given up when he might have captured the victor’s cup;
  and he learned too late when the night came down,
  how close he was to the golden crown.
  Success is failure turned inside out – the silver tint of the clouds of doubt,
  and when you never can tell how close you are,
  it may be near when it seems afar;
  so stick to the fight when you’re hardest hit – it’s when things seem worst, you must not quit.
"""
model.predict(clean(poem))

['Don’t Quit']

In [ ]:
# Mother to Son
poem = """
  Well, son, I’ll tell you:
  Life for me ain’t been no crystal stair.
  It’s had tacks in it,
  And splinters,
  And boards torn up,
  And places with no carpet on the floor—
  Bare;
  But all the time
  I’se been a’climbin’ on,
  And reachin’ landin’s,
  And turnin’ corners,
  And sometimes goin’ in the dark,
  Where there ain’t been no light.
  So boy, don’t you turn back;
  Don’t you sit down on the steps,
  ‘Cause you finds it’s kinder hard;
  Don’t you fall now—.
  For I’se still goin’, honey,
  I’se still climbin’,
  And life for me ain’t been no crystal stair.
"""
model.predict(clean(poem))

['life for me hasnt been no crystal stair']

In [ ]:
# Immortality
poem = """
  To be forever young
  and ride like a tipsy Triton
  on the crest of a wave
  that is just forever breaking. . . .
  Days—an eternal dawning
  heralded with the fanfare of sun,
  Nights—a blaze of glory
  the swishing tail of a comet,
  Life—an infinite loving
  Sweeping to the peak of anticipation
  Trembling breathlessly at the brink of realization. . . .
"""
model.predict(clean(poem))

['To be forever young and ride like a tipsy triton on the crest of a wave']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# It's advisable to create a new python environment and install simplet5
!pip install --upgrade simplet5

In [ ]:
def clean(input_string):
    import string
    import re

    punctuation_chars = set(string.punctuation)
    cleaned_string = ''.join(char for char in input_string if char not in punctuation_chars)
    cleaned_string = cleaned_string.replace('\n', '')
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string)
    return cleaned_string.lower()

def generate_title(poem):
  from simplet5 import SimpleT5

  # instantiate
  model = SimpleT5()
  model.load_model("t5","/content/drive/MyDrive/Poetry/simplet5-epoch-4-train-loss-nan-val-loss-2.9005", use_gpu=True)

  return model.predict(clean(poem))[0]

In [ ]:
poem = """
  When things go wrong, as they sometimes will,
  when the road you’re trudging seems all uphill,
  when the funds are low and the debts are high,
  and you want to smile but you have to sigh,
  when care is pressing you down a bit – rest if you must, but don’t you quit.
  Life is queer with its twists and turns.
  As everyone of us sometimes learns.
  And many a fellow turns about when he might have won had he stuck it out.
  Don’t give up though the pace seems slow – you may succeed with another blow.
  Often the goal is nearer than it seems to a faint and faltering man;
  Often the struggler has given up when he might have captured the victor’s cup;
  and he learned too late when the night came down,
  how close he was to the golden crown.
  Success is failure turned inside out – the silver tint of the clouds of doubt,
  and when you never can tell how close you are,
  it may be near when it seems afar;
  so stick to the fight when you’re hardest hit – it’s when things seem worst, you must not quit.
"""
# Don’t Quit
generate_title(poem)

'Don’t Quit'